📝 **Author:** Amirhossein Heydari - 📧 **Email:** amirhosseinheydari78@gmail.com - 📍 **Linktree:** [linktr.ee/mr_pylin](https://linktr.ee/mr_pylin)

---

# Dependencies

In [1]:
import torch
from torch import nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchinfo import summary
from torchvision.datasets import FakeData
from torchvision.models import AlexNet
from torchvision.transforms import v2 as transforms

# Parameters
   - Parameters are the core elements that define the model's behavior and functionality.
   - These parameters are **learned** from the **training data** and are crucial for making accurate predictions.
   - The primary parameters in neural networks are **weights** and **biases**.
      - **Weights** determine the strength of the connection between neurons
      - **Biases** allow the model to shift the activation function to better fit the data.

In [2]:
# initialize AlexNet with random weights and biases
model = AlexNet(num_classes=1000)
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
summary(model=model, input_size=(1, 3, 227, 227), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 56, 56]           23,296
│    └─ReLU: 2-2                         [1, 64, 56, 56]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

# Hyperparameters
   - Hyperparameters in deep learning models are settings that you configure **before training** your model.
   - Hyperparameters are **not learned** from the **data** but are crucial for controlling the **training process** and **model architecture**.

## A List of Hyperparameters
<table style="margin: 0 auto;">
   <tbody>
      <tr>
         <td>Train-Validation-Test Ratio</td>
         <td>Data Augmentation</td>
         <td>Normalizations</td>
         <td>Weight Initialization</td>
         <td>Number of Layers</td>
         <td>Number of Neurons</td>
      </tr>
      <tr>
         <td>Activation Functions</td>
         <td>Loss Function</td>
         <td>Optimizer</td>
         <td>Learning Rate</td>
         <td>Learning Rate Decay</td>
         <td>Momentum</td>
      </tr>
      <tr>
         <td>Batch Size</td>
         <td>Number of Epochs</td>
         <td>Dropout Rate</td>
         <td>Regularization</td>
         <td>Gradient Clipping</td>
         <td>Early Stopping</td>
      </tr>
   </tbody>
</table>

## Train-Validation-Test Ratio
   - The Train-Validation-Test Ratio is the proportion in which the dataset is split into three subsets:
      - **Training Set**: Used to train the model.
      - **Validation Set**: Used to tune hyperparameters and evaluate the model during training.
      - **Test Set**: Used to evaluate the final model performance.

**✍️ Key Points**
   - A larger **training** set can help the model learn better, **but** it should not be so large that the **validation** and **test** sets are too small to provide reliable evaluations.
   - A properly sized **validation** set helps in **tuning hyperparameters** and **preventing overfitting**.
   - A sufficiently large **test** set ensures that the final evaluation of the model is **reliable** and **unbiased**.

In [4]:
# create an artificial dataset
dataset = FakeData(size=5000, image_size=(3, 32, 32), num_classes=3, transform=None)

# define the train-validation-test split ratios
train_ratio = 0.7
val_ratio   = 0.15
test_ratio  = 0.15

# split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_ratio, val_ratio, test_ratio])

In [5]:
# extract labels
targets = torch.tensor([l for _, l in dataset])

# calculate distribution of each set
train_distribution = dict(zip(*[c.tolist() for c in torch.unique(targets[train_dataset.indices], return_counts=True)]))
val_distribution   = dict(zip(*[c.tolist() for c in torch.unique(targets[val_dataset.indices]  , return_counts=True)]))
test_distribution  = dict(zip(*[c.tolist() for c in torch.unique(targets[test_dataset.indices] , return_counts=True)]))

# log
print("train_dataset:")
print(f"\t -> len(train_dataset) : {len(train_dataset)}")
print(f"\t -> distibution        : {train_distribution}\n")
print("val_dataset:")
print(f"\t -> len(val_dataset)   : {len(val_dataset)}")
print(f"\t -> distibution        : {val_distribution}\n")
print("test_dataset:")
print(f"\t -> len(test_dataset)  : {len(test_dataset)}")
print(f"\t -> distibution        : {test_distribution}")

train_dataset:
	 -> len(train_dataset) : 3500
	 -> distibution        : {0: 1143, 1: 1159, 2: 1198}

val_dataset:
	 -> len(val_dataset)   : 750
	 -> distibution        : {0: 262, 1: 248, 2: 240}

test_dataset:
	 -> len(test_dataset)  : 750
	 -> distibution        : {0: 260, 1: 243, 2: 247}


## Data Augmentation
   - Data augmentation is a technique used to **artificially increase** the size of a **training** dataset by creating **modified** versions of the data.
   - This helps improve the model's ability to **generalize** by providing more **varied** training examples.
   - Common data augmentation techniques include **rotations**, **translations**, **flips**, and **color adjustments**.

**✍️ Key Points**
   - **Improves Generalization**: By exposing the model to a wider **variety** of data, it can learn more **robust** features and perform better on **unseen data**.
   - **Reduces Overfitting**: Augmented data helps prevent the model from **memorizing the training data**, thus reducing **overfitting**.

In [6]:
# define transformations including data augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),                    # randomly flips the image horizontally with a 50% chance
    transforms.RandomRotation(degrees=10),                # randomly rotates the image by up to 10 degrees
    transforms.ToImage(),                                 # convert the tensor to an image
    transforms.ToDtype(dtype=torch.float32, scale=True),  # convert the image to float32 and scale it
    transforms.Normalize(mean=(0.5,), std=(0.5,))         # normalize the image
])

# load the dataset with the defined transformations
dataset = FakeData(size=5000, image_size=(3, 32, 32), num_classes=3, transform=transform)

# log
print(dataset.extra_repr)

<bound method VisionDataset.extra_repr of Dataset FakeData
    Number of datapoints: 5000
    StandardTransform
Transform: Compose(
                 RandomHorizontalFlip(p=0.5)
                 RandomRotation(degrees=[-10.0, 10.0], interpolation=InterpolationMode.NEAREST, expand=False, fill=0)
                 ToImage()
                 ToDtype(scale=True)
                 Normalize(mean=[0.5], std=[0.5], inplace=False)
           )>


## Batch Size
   - Batch size is a hyperparameter that defines the **number of training examples** used in one **iteration**.
   - It determines how many samples are processed before the model's internal parameters are **updated**.

**✍️ Key Points**
   - **Small Batch Size**: Can lead to more **noisy** updates but can help the model **generalize** better. It also requires **less memory**.
   - **Large Batch Size**: Can speed up training by making more efficient use of hardware but may lead to **overfitting** and requires **more memory**.

In [7]:
# create an artificial dataset
data = torch.randn(1000, 10)           # 1000 samples, each with 10 features
labels = torch.randint(0, 2, (1000,))  # binary labels (0 or 1) for each sample

# combine data and labels into a TensorDataset
dataset = TensorDataset(data, labels)

# define different batch sizes
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for batch_idx, (inputs, targets) in enumerate(dataloader):
    outputs = inputs.sum(dim=1)
    print(outputs.shape)

torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([104])


## Weight Initialization
   - Weight initialization is the process of setting the initial values of the **weights** in a neural network **before** training begins.

**✍️ Key Points**
   - **Faster Convergence**: Proper initialization can lead to faster convergence by providing a **good starting point** for the optimization process.
   - **Stability**: Helps in stabilizing the training process by **preventing large updates** to the weights.
   - **Avoiding Vanishing/Exploding Gradients**: Proper initialization can prevent the gradients from becoming too small (**vanishing**) or too large (**exploding**) during **backpropagation**.

**📈 Common Initialization Techniques for Weights**
   1. **Zero Initialization**
      - All weights are initialized to zero.
      - Not recommended for deep networks as it can lead to symmetry problems where all neurons in a layer learn the same features.
   
   1. **Random Initialization**
      - Weights are initialized randomly, typically from a uniform or normal distribution.
      - Provides a diverse set of starting points but can still lead to issues with vanishing or exploding gradients.
   
   1. **Xavier (Glorot) Initialization**
      - Weights are initialized from:
         - $W \sim \mathcal{N}\left(0, \frac{2}{n_{\text{in}} + n_{\text{out}}}\right)$.
         - $W \sim \mathcal{U}\left(-{gain}\times\sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}, {gain}\times\sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}\right)$.
      - Helps in maintaining the variance of activations and gradients throughout the network
   
   1. **He (Kaiming) Initialization**
      - Weights are initialized from:
         - $W \sim \mathcal{N}\left(0, \frac{2}{n_{\text{in}}}\right)$.
         - $W \sim \mathcal{U}\left(-{gain}\times\sqrt{\frac{6}{n_{\text{in}}}}, {gain}\times\sqrt{\frac{6}{n_{\text{in}}}}\right)$.
      - Particularly useful for networks with ReLU activations as it helps in maintaining the variance of activations

**📉 Common Initialization Techniques for Biases**
   1. **Zero Initialization**
      - All biases are initialized to zero.
      - Generally works well and is commonly used because it does not introduce any initial bias in the learning process.
   
   1. **Constant Initialization**
      - All biases are initialized to a constant value, often a small positive value like 0.01.
      - Can help in ensuring that all neurons in a layer start with a small positive bias, which can be beneficial in some cases.

In [8]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 2)
        self._initialize_weights()
    
    def _initialize_weights(self):
        # initializing weights based on the xavier formula (normal distribution version)
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        
        # initializing biases with zero
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
    
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

# other initializations
# nn.init.uniform_
# nn.init.zeros_
# nn.init.normal_
# nn.init.xavier_uniform_
# nn.init.kaiming_normal_
# nn.init.kaiming_uniform_

## Number of Layers & Neurons
   - The number of layers and neurons in a neural network defines its **architecture**.
   - **Number of Layers** Refers to the **depth** of the network.
   - **Number of Neurons** Refers to the **width** of each layer.

**✍️ Key Points**
   - **Model Capacity**: Increasing the number of layers and neurons increases the model's capacity to learn **complex patterns**.
   - **Overfitting**: Too many layers and neurons can lead to **overfitting**, where the model performs well on training data but poorly on **unseen data**.
   - **Computational Cost**: More layers and neurons increase the **computational cost** and **training time**.

In [9]:
# define a complex neural network with more layers and neurons
class ComplexNN(nn.Module):
    def __init__(self):
        super(ComplexNN, self).__init__()
        self.fc1 = nn.Linear(10, 100)
        self.fc2 = nn.Linear(100, 200)
        self.fc3 = nn.Linear(200, 100)
        self.fc4 = nn.Linear(100, 50)
        self.fc5 = nn.Linear(50, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

## Normalizations
   - Normalization techniques in neural networks are used to **standardize** the inputs to a layer, improving the **training speed** and **stability**.
   - Common normalization techniques include **Batch Normalization**, **Layer Normalization**, **Instance Normalization**, and **Group Normalization**.

**✍️ Key Points**
   - **Faster Convergence**: Normalization helps in **faster convergence** by reducing **internal covariate** shift.
   - **Improved Performance**: Ensures that the model learns more effectively by providing standardized inputs.
   - **Stability**: Helps in stabilizing the training process by preventing large gradient updates.

In [10]:
# define a simple neural network with Batch Normalization
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

## Activation Functions
   - Activation functions introduce **non-linearity** into the neural network, enabling it to learn and represent **complex patterns** in the data.
   - They determine the output of a neuron given an input or set of inputs.
   - Choosing the right activation function is crucial for the performance of the neural network.
   - **More info**: [01_activation-functions.ipynb](./01_activation-functions.ipynb)

In [11]:
# define a custom neural network that uses several activation functions
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, 10)
        self.fc5 = nn.Linear(10, 5)
        self.fc6 = nn.Linear(5, 3)
    
        # define activation functions
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU(0.01)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.leaky_relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.softmax(self.fc6(x))
        return x

## Loss Function
   - Loss functions (aka **cost/objective** functions), measure how well a neural network's predictions match the actual target values.
   - They guide the optimization process by providing a measure of the model's performance.
   - The choice of loss function depends on the type of problem being solved (e.g., **regression**, **binary classification**, **multi-class classification**)
   - **More info**: [02_loss-functions.ipynb](./02_loss-functions.ipynb)

In [12]:
# regression example

# artificial true and predicted values
y_true = torch.tensor([2.5, 0.0, 2.1, 7.8])
y_pred = torch.tensor([3.0, -0.5, 2.0, 8.0])

# define the loss function
criterion = nn.MSELoss()

# compute the loss
loss = criterion(y_pred, y_true)
print(f'MSE: {loss.item()}')

MSE: 0.13749997317790985


In [13]:
# binary classification example

# artificial true and predicted values
y_true = torch.tensor([1, 0, 1, 0], dtype=torch.float32)
y_pred = torch.tensor([0.9, 0.1, 0.8, 0.4], dtype=torch.float32)

# define the loss function
criterion = nn.BCELoss()

# compute the loss
loss = criterion(y_pred, y_true)
print(f'BCE: {loss.item()}')

BCE: 0.23617255687713623


In [14]:
# multi-class classification example

# artificial true and predicted values
y_true = torch.tensor([2, 0, 1])
y_pred = torch.tensor([[0.1, 0.2, 0.7],
                       [0.8, 0.1, 0.1],
                       [0.2, 0.6, 0.2]])

# define the loss function
criterion = nn.CrossEntropyLoss()

# compute the loss
loss = criterion(y_pred, y_true)
print(f'CE: {loss.item()}')

CE: 0.7693669199943542


## Optimizer
   - Optimizers are algorithms used to **update the weights** of a neural network to **minimize** the loss function.
   - They determine how the model's parameters are adjusted based on the gradients computed during **backpropagation**.
   - Different optimizers impact the **convergence speed** and **final performance** of the model.

**Common Optimizers**:
   - Stochastic Gradient Descent (SGD)
   $$ \theta = \theta - \eta \nabla J(\theta) $$
   
   - Adam (Adaptive Moment Estimation)
   $$ m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t $$
   $$ v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2 $$
   $$ \hat{m}_t = \frac{m_t}{1 - \beta_1^t} $$
   $$ \hat{v}_t = \frac{v_t}{1 - \beta_2^t} $$
   $$ \theta = \theta - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} $$
   
   - RMSProp (Root Mean Square Propagation)
   $$ E[g^2]t = \gamma E[g^2]{t-1} + (1 - \gamma) g_t^2 $$
   $$ \theta = \theta - \frac{\eta}{\sqrt{E[g^2]_t + \epsilon}} g_t $$

In [15]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# create a model instance
model = SimpleNN()

# define the optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# example input and target
input = torch.randn(5, 10)
target = torch.randn(5, 1)

# forward pass
output = model(input)

# compute the loss
criterion = nn.MSELoss()
loss = criterion(output, target)

# backward pass and optimization
optimizer.zero_grad()
loss.backward()
optimizer.step()

## Learning Rate
   - The learning rate is a hyperparameter that controls the step size at each iteration while moving toward a minimum of the loss function.
   - It determines how much to change the model's parameters in response to the estimated error each time the model weights are updated.

**✍️ Key Points**
   - **High Learning Rate**: Can cause the model to **converge** too quickly to a **suboptimal solution** or even **diverge**.
   - **Low Learning Rate**: Can make the training process very **slow** and may get stuck in **local minima**.

In [16]:
model = AlexNet()
optimizer_1 = SGD(params=model.parameters(), lr=0.1)
optimizer_2 = Adam(params=model.parameters(), lr=0.001)

# log
print(optimizer_1)
print(optimizer_2)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


## Momentum
   - Momentum is a technique used to **accelerate** the convergence of the optimization process by **adding a fraction of the previous update to the current update**.
   - It helps in **smoothing** the optimization path and can **prevent** the model from getting stuck in **local minima**.

**✍️ Key Points**
   - **Faster Convergence**: Helps in accelerating the convergence by smoothing the optimization path.
   - **Stability**: Reduces oscillations and helps in stabilizing the training process.

In [17]:
model = AlexNet()
optimizer = SGD(params=model.parameters(), lr=0.1, momentum=0.5)

# log
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)


## Number of Epochs
   - It defines how many times the learning algorithm will work through the **entire** training dataset.

**✍️ Key Points**
   - More epochs can lead to **better learning**, but too many can cause **overfitting**.
   - The right number of epochs helps the model **generalize well** to new data.

In [18]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# generate artificial data
data = torch.randn(1000, 10)  # 1000 samples, 10 features each
labels = torch.randint(0, 2, (1000,))  # Binary labels (0 or 1)

# create DataLoader
dataset = TensorDataset(data, labels)
trainloader = DataLoader(dataset, batch_size=64, shuffle=True)

# initialize network, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01)

# epochs
epochs = 5

# training loop
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"epoch {epoch+1}, loss: {running_loss/len(trainloader)}")

epoch 1, loss: 0.6985896341502666
epoch 2, loss: 0.6910563744604588
epoch 3, loss: 0.6890373341739178
epoch 4, loss: 0.6882805302739143
epoch 5, loss: 0.6871195696294308


## Learning Rate Decay
   - Learning rate decay is a technique used to reduce the learning rate over time during training.
   - This helps the model converge more precisely by taking smaller steps as it approaches the minimum of the loss function.
   - Learning rate decay can be implemented in various ways, such as step decay, exponential decay, and adaptive learning rates.

**✍️ Key Points**
   - **Improved Convergence**: Helps the model converge more precisely by reducing the learning rate over time.
   - **Stability**: Reduces the risk of overshooting the minimum of the loss function by taking smaller steps as training progresses.

In [19]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# create a model instance
model = SimpleNN()
criterion = nn.MSELoss()
optimizer = SGD(model.parameters(), lr=0.1)

# define a learning rate scheduler
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

# example input and target
input = torch.randn(5, 10)
target = torch.randn(5, 1)

# mimic the training loop with learning rate decay
for epoch in range(10):
    # forward pass
    output = model(input)

    # compute the loss
    loss = criterion(output, target)

    # backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # step the learning rate scheduler
    scheduler.step()

    print(f'Epoch {epoch+1:>2}, Loss: {loss.item():.17f}, Learning Rate: {scheduler.get_last_lr()[0]}')

Epoch  1, Loss: 1.23805165290832520, Learning Rate: 0.1
Epoch  2, Loss: 0.60764205455780029, Learning Rate: 0.1
Epoch  3, Loss: 0.40673893690109253, Learning Rate: 0.010000000000000002
Epoch  4, Loss: 0.26944941282272339, Learning Rate: 0.010000000000000002
Epoch  5, Loss: 0.25881299376487732, Learning Rate: 0.010000000000000002
Epoch  6, Loss: 0.24861660599708557, Learning Rate: 0.0010000000000000002
Epoch  7, Loss: 0.23877961933612823, Learning Rate: 0.0010000000000000002
Epoch  8, Loss: 0.23781020939350128, Learning Rate: 0.0010000000000000002
Epoch  9, Loss: 0.23684391379356384, Learning Rate: 0.00010000000000000003
Epoch 10, Loss: 0.23588076233863831, Learning Rate: 0.00010000000000000003


## Dropout Rate
   - It sets a **fraction of input units** to **zero** at each update during training time, which forces the network to learn more **robust features**.
   - Helps in regularizing the model and preventing **overfitting** by ensuring that the network does not rely too heavily on **any individual neuron**.

In [20]:
class SimpleNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(10, 128)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## Regularization
   - Regularization is a technique used to prevent **overfitting** by adding **a penalty to the loss function**.
   - This penalty **discourages** the model from fitting **too closely** to the training data, which helps improve its **generalization** to new data.

✍️ **Common Regularizations**
   - **L1 (Lasso) Regularization**:
      - Adds the **absolute** value of the **coefficients** as a penalty term to the **loss function**.
      - $ L1 = \lambda \sum_{i=1}^{n} |w_i| $
   - **L2 (Ridge) Regularization**:
      - Adds the **squared** value of the **coefficients** as a penalty term to the **loss function**.
      - $ L2 = \lambda \sum_{i=1}^{n} w_i^2 $

In [21]:
# training function + l1 regularization
def train_model(epochs, l1_lambda=0.01):
    # initialize loss function, and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01)
    
    for epoch in range(function):
        for inputs, targets in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            
            # add L1 regularization to the loss
            loss = criterion(outputs, targets)
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            loss = loss + l1_lambda * l1_norm
            
            loss.backward()
            optimizer.step()

In [22]:
# training function + l2 regularization
def train_model(epochs, l2_lambda=0.01):
    # initialize loss function, and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, weight_decay=l2_lambda)  # weight_decay is the coefficient for l2_norm
    
    for epoch in range(epochs):
        for inputs, targets in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

## Gradient Clipping
   - It is a technique used to **prevent the exploding gradient** problem in neural networks, especially in **recurrent neural networks (RNNs)**.
   - It involves **capping the gradients** during backpropagation to a **maximum** value to ensure they don't become **too large**.

In [23]:
# training function + gradient clipping
def train_model(epochs, clip_value=1.0):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01)
    
    for epoch in range(epochs):
        for inputs, targets in trainloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, targets)
            loss.backward()
            
            # apply gradient clipping
            nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            
            optimizer.step()

## Early Stopping
   - It is used to **prevent overfitting** by **stopping** the training process when the model's performance on a **validation set** starts to degrade.

In [24]:
def train_model(epochs, trainloader, valloader, patience=3):
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        
        # train loop
        model.train()
        train_loss = 0.0
        for inputs, targets in trainloader:
            pass
        
        # validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in valloader:
                pass
        
        # check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break